数据默认放在notebook的同目录的data文件下


In [56]:
import pandas as pd, numpy as np, matplotlib.pylab as plt

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    fig.set_size_inches(10,10)
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           title=title,
           ylabel='True label',
           xlabel='Predicted label')
    ax.set_xticklabels(classes[0],fontsize=20)
    ax.set_yticklabels(classes[1],fontsize=20)
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt), fontsize =30,
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


In [3]:
#读取数据
df = pd.read_csv('data/train.csv')

#不需要id code
df = df.drop('ID_code',axis = 1)

In [6]:
df.head(5)



,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [11]:
from sklearn.model_selection import StratifiedShuffleSplit
#分测试训练集
def train_test_split(df, target, percentage=0.3):
    X = df.drop([target],axis = 1)
    y = df[target]
    sss = StratifiedShuffleSplit(n_splits=1, test_size=percentage)
    for train_idx, test_idx in sss.split(X,y):
        df_test = df.loc[test_idx]
        df_train = df.loc[train_idx]
    df_train.index = range(len(df_train))
    df_test.index = range(len(df_test))
    return df_train, df_test
    

#如需更改训练测试比例，请在此处更改

fraction = 0.3

df_train, df_test = train_test_split(df, 'target', fraction)

In [45]:
df = SMOTE_methods(df_train, 'target', 'regular')

In [44]:
#采样处理函数区
#smote需中心化处理
from imblearn.combine import SMOTEENN
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
def SMOTE_methods(df_train,target,method):
    '''The output data has been normalized by MinMaxScaler'''
    scaler = MinMaxScaler()
    X = df_train.drop([target],axis=1)
    y = df_train[target]
    X_normalized = scaler.fit_transform(X)
    if method == 'regular':
        X_res, y_res = SMOTE(kind='regular').fit_sample(X_normalized, y)
    elif method == 'borderline1':
        X_res, y_res = SMOTE(kind='borderline1').fit_sample(X_normalized, y)
    elif method == 'borderline2':
        X_res, y_res = SMOTE(kind='borderline2').fit_sample(X_normalized, y)
    elif method == 'svm':
        X_res, y_res = SMOET(kind='svm').fit_sample(X_normalized, y)
    elif method == 'Tomek':
        sm = SMOTETomek()
        X_res, y_res = sm().fit_sample(X_normalized, y)
    elif method == 'ENN':
        sm = SMOTEENN()
        X_res, y_res = sm().fit_sample(X_normalized, y)
    else:
        raise ValueError('输入方法有误')
    df_final = pd.DataFrame(X_res, columns = X.columns)
    df_final['target'] = y_res
    return df_final

#为方便之后测试，请务必使用函数形式

def under_sample(df_train):
    '''
    input: training dateframe including target
    output: under sampled dataframe
    '''
    t0 = df_train[df_train['target']==0]
    t1 = df_train[df_train['target']==1]


    df = t0.sample(frac=len(t1)/len(t0)).append(t1)
    
    return df

def over_sample(df_train):
    '''
    input: training dateframe including target
    output: under sampled dataframe
    '''
    t0 = df_train[df_train['target']==0]
    t1 = df_train[df_train['target']==1]


    df = t1.sample(frac=len(t0)/len(t1),replace=True).append(t0)
    
    return df



In [51]:
#（一般情况下不需要更改）

df_train_sampled = under_sample(df_train)



print ('数据总共有%s个， 其中0标签%s个， 1标签%s个'%( len(df_train_sampled),sum(df_train_sampled['target']==0),sum(df_train_sampled['target']==1)))

数据总共有28138个， 其中0标签14069个， 1标签14069个


In [52]:
#分开标签 (一般情况下不需要更改)
t_train = df_train_sampled['target']
x_train = df_train_sampled.drop('target',axis=1)

t_test = df_test['target']
x_test = df_test.drop('target',axis=1)

In [75]:
#变量处理



In [36]:
#变量晒选



12683

In [54]:
#初步建模
%timeit
#xgb
from xgboost import XGBClassifier
model = XGBClassifier(
 learning_rate =0.1,
 n_estimators=100,
 max_depth=9,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 scale_pos_weight=1,
 seed=27)

model.fit(x_train,t_train)



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=9, min_child_weight=5, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
       subsample=0.8)

In [57]:
#模型检验
t_pred = model.predict(x_test)

# Plot normalized confusion matrix
plot_confusion_matrix(t_test, t_pred, classes=[u'TF',u'TF'], normalize=True, title='Normalized XGboost confusion matrix')
plt.show()

#因子重要性
from xgboost import plot_importance
fig,ax = plt.subplots(figsize=(10,15))
ax.set_yticklabels(x_train.columns,fontsize= 15)
plot_importance(model,height=0.5,max_num_features=20,ax=ax)
plt.show()

#ROCAUC
t_score = roc_auc_score(t_test,model.predict_proba(x_test)[:,1])
print('AUC score: %s'%(t_score))

fpr, tpr, thresholds= roc_curve(t_test,model.predict_proba(X_test)[:,1],pos_label=None,sample_weight=None,drop_intermediate=True)
plt.plot(fpr,tpr,marker = 'o')
plt.title('ROC')
plt.show()



NameError: name 'confusion_matrix' is not defined